**1. Import**

In [ ]:
# Import

import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split, WeightedRandomSampler
import timm
from torch.cuda.amp import autocast, GradScaler
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import Counter
from sklearn.metrics import precision_score, recall_score, f1_score
from torch.utils.tensorboard import SummaryWriter
import os


In [ ]:
import random
import numpy as np

# Set Seed for Reproducibility
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)


**2. Device & Data Transform**

In [ ]:
# Set device & Set Device and Define Data Transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Transforms
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

**3. Load Dataset And Class Distribution**

In [ ]:
# Load Dataset and Analyze Class Distribution

# Dataset path
DATASET_PATH = "/kaggle/input/ct-kidney-dataset-normal-cyst-tumor-and-stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone"

# Load dataset
dataset = datasets.ImageFolder(root=DATASET_PATH, transform=transform_train)
print("Classes:", dataset.classes)

# Class distribution
labels = [sample[1] for sample in dataset.samples]
label_counts = Counter(labels)
print("Class distribution:", label_counts)

**4. Split Dataset & Create Balanced Sampler**

In [ ]:
# Split dataset first
total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.2 * total_size)
test_size = total_size - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Apply transform_test to val and test
val_dataset.dataset.transform = transform_test
test_dataset.dataset.transform = transform_test

# Compute class weights ONLY for train_dataset
train_labels = [dataset[i][1] for i in train_dataset.indices]
train_counts = Counter(train_labels)
class_weights = [1.0 / train_counts[label] for label in train_labels]
sampler = WeightedRandomSampler(class_weights, num_samples=len(train_labels), replacement=True)


**Split sizes**

In [ ]:
# Verify split sizes
print("Train Dataset Size:", len(train_dataset))
print("Validation Dataset Size:", len(val_dataset))
print("Test Dataset Size:", len(test_dataset))

**5. Create DataLoaders for Training, Validation, and Testing**

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


**6. Swin Transformer**

In [ ]:
# Swin Transformer Model
class SwinClassifier(nn.Module):
    def __init__(self, num_classes):
        super(SwinClassifier, self).__init__()
        self.model = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True, num_classes=num_classes)

    def forward(self, x):
        return self.model(x)


**7. Initialize Model, Loss Function, Optimizer, and Scaler**

In [ ]:
# Initialize model
num_classes = len(dataset.classes)
model = SwinClassifier(num_classes=num_classes).to(device)

# Optimizer and loss
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)
scaler = GradScaler()

**8. Training Function with AMP and Early Stopping**

In [ ]:
# Training function with AMP
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=300):
    best_val_loss = float('inf')
    patience = 3
    counter = 0
    train_losses, val_losses = [], []

    for epoch in range(epochs):
        model.train()
        total_loss, correct = 0, 0
        
        # === Training Loop ===
        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1} Training"):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            # Automatic Mixed Precision
            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()

        train_loss = total_loss / len(train_loader.dataset)
        train_acc = 100. * correct / len(train_loader.dataset)
        train_losses.append(train_loss)

        # Validation
        model.eval()
        total_val_loss, val_correct = 0, 0
        all_preds, all_labels = [], []

        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc=f"Epoch {epoch+1} Validation"):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                total_val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_loss = total_val_loss / len(val_loader.dataset)
        val_acc = 100. * val_correct / len(val_loader.dataset)
        val_losses.append(val_loss)

        precision = precision_score(all_labels, all_preds, average='macro')
        recall = recall_score(all_labels, all_preds, average='macro')
        f1 = f1_score(all_labels, all_preds, average='macro')

        print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}, Train Acc={train_acc:.2f}%, Val Acc={val_acc:.2f}%")
        print(f"Precision={precision:.4f}, Recall={recall:.4f}, F1 Score={f1:.4f}")

        writer.add_scalar("Loss/Train", train_loss, epoch)
        writer.add_scalar("Loss/Val", val_loss, epoch)
        writer.add_scalar("Accuracy/Train", train_acc, epoch)
        writer.add_scalar("Accuracy/Val", val_acc, epoch)
        writer.add_scalar("Precision/Val", precision, epoch)
        writer.add_scalar("Recall/Val", recall, epoch)
        writer.add_scalar("F1/Val", f1, epoch)
        
        # === Early Stopping & Save Best Model ===
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_epoch = epoch + 1
            counter = 0
            torch.save(model.state_dict(), '/kaggle/working/best_model.pth')

        else:
            counter += 1
            if counter >= patience:
                print("Early stopping triggered.")
                break
    print(f"\n✅ Best Model was from Epoch {best_epoch} with Val Loss: {best_val_loss:.4f}")
    
    # Plot loss
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.legend()
    plt.title("Loss Curve")
    plt.show()
    


**9. TensorBoard Logger**

In [ ]:
writer = SummaryWriter(log_dir="/kaggle/working/runs/swin_ct_kidney")

**10. Training**

In [ ]:
# Start training
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=300)

In [ ]:
writer.close()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs


In [ ]:
 %reload_ext tensorboard

**11. Evaluation on Test Set**

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Testing"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Compute metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    print(f"\nTest Accuracy: {accuracy:.4f}")
    print(f"Test Precision: {precision:.4f}")
    print(f"Test Recall: {recall:.4f}")
    print(f"Test F1 Score: {f1:.4f}")

# Evaluate on test set
evaluate_model(model, test_loader)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

def evaluate_model(model, test_loader, class_names=None):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Testing"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # === Overall Metrics ===
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"\nOverall Accuracy: {accuracy:.4f}")

    # === Per-Class Metrics ===
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names, digits=4))


**12. Classification Report**

In [ ]:
class_names = ['Cyst', 'Normal', 'Stone', 'Tumor']
evaluate_model(model, test_loader, class_names)